In [1]:
!pip install pandas transformers


In [6]:
pip install torch torchvision torchaudio

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/1e/86/477ec85bf1f122931f00a2f3889ed9322c091497415a563291ffc119dacc/torch-2.1.2-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/ef/a2/f16cac894c4c71585b3411707502ed8d607945fb4a695857621565bd728d/torchvision-0.16.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/42/e6/cd2386ff0bb1476c472593a77ca1870cbbddef901e63b04cc84ccd368da7/torchaudio-2.1.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import json
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# transformers 라이브러리와 pandas 라이브러리가 설치되어 있는지 확인하세요:
# !pip install transformers pandas

# 모델과 토크나이저 초기화
model_name = "snunlp/KR-Medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 각 파일에 대해 감성 분석을 수행하고 JSON으로 결과를 저장하는 함수
def perform_sentiment_analysis(file_path, columns, file_type='csv', result_file_path=None):
    # 파일 타입에 따라 다르게 읽기
    if file_type == 'csv':
        df = pd.read_csv(file_path)
    elif file_type == 'json':
        df = pd.read_json(file_path)
    
    print(df.head())  # 데이터의 상위 몇 줄을 출력하여 확인
    
    # 결과 파일 이름 설정
    if result_file_path is None:
        result_file_path = file_path.split('.')[0] + '_sentiment_results.json'
    
    sentiment_results = []  # 감성 분석 결과를 저장할 리스트

    # 각 행과 지정된 컬럼에 대한 감성 분석을 수행
    for _, row in df.iterrows():
        for column in columns:
            text = row.get(column, '')
            if text:
                try:
                    sentiment = nlp(text)[0]
                    sentiment_results.append({
                        "text": text,
                        "column": column,
                        "sentiment_label": sentiment['label'],
                        "sentiment_score": sentiment['score']
                    })
                except Exception as e:
                    print(f"텍스트 처리 오류: {text}. 오류: {e}")
    
    # 결과를 JSON 파일로 저장
    with open(result_file_path, 'w', encoding='utf-8') as file:
        json.dump(sentiment_results, file, ensure_ascii=False, indent=4)
    
    print(f"결과가 {result_file_path}에 저장되었습니다")

# 각 파일에 대해 함수를 호출합니다.
# 각 파일에 대해 함수를 호출합니다. 경로를 정확하게 지정하세요.
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/combined_data.csv', ['title', 'contents'])
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/inti_op.csv', ['text'])
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/naver_news.json', ['title', 'description'], file_type='json')
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/youtube_comments.csv', ['comment'])
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/youtube_videos.csv', ['video_title', 'video_des'])




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-Medium and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    id                                    Title  \
0  201                        김포 서울편입은 흐지부지 된거냐   
1  202                       서울에도 메트로개념 하면 안되려나   
2  203                  김포 서울 편입이슈가 한주도 안되서 잊혀짐   
3  204                      김포 서울 편입되면 얻게 되는 효과   
4  205  조경태 “메가시티는 제2새마을운동...국가부채 1000조, 뭉쳐야 산다   

                                           Post Link  \
0  https://gall.dcinside.com/board/view/?id=drama...   
1  https://gall.dcinside.com/mgallery/board/view/...   
2  https://gall.dcinside.com/mgallery/board/view/...   
3  https://gall.dcinside.com/board/view/?id=immov...   
4  https://gall.dcinside.com/mgallery/board/view/...   

                                            Contents  
0                                         갑자기 조용해진 듯  
1  요즘 서울 확장 얘기 많던데 서울"시"로 편입하기보다는 메트로벤쿠버나 GTA, 혹은...  
2              총선까지 한참 남았는데 병신들만 남아있는게 아닌가 생각이 된다 ㄹㅇ  
3  1. 각종 기피시설 김포 이전2. 김포집값이 서울아파트 평균값을 떨어뜨린다3. 서울...  
4  (주간조선)조경태 “메가시티는 제2 새마을운동...국가부채 1000조, 뭉쳐야 산다...  
결과가 /Users/jeonghyeonjeong

텍스트 처리 오류: nan. 오류: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
텍스트 처리 오류: nan. 오류: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
텍스트 처리 오류: nan. 오류: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
텍스트 처리 오류: nan. 오류: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
텍스트 처리 오류: 롬<a href="https://www.youtube.com/watch?v=78OkXyP4ri0&amp;t=3m23s">3:23</a><br>이는 모든 사람이 죄를 지었으므로<br>하나님의 영광에 이르지 못하다가.<br><br>롬<a href="https://www.youtube.com/watch?v=78OkXyP4ri0&amp;t=6m23s">6:23</a><br>죄의 삯은 사망이요.<br><br>죄사함을 받지 못하시면 영원토록<br>불타는 지옥에서 고통받습니다<br><br>행 <a href="https://www.y

KeyboardInterrupt: 

In [3]:
import pandas as pd
import json
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# transformers 라이브러리와 pandas 라이브러리가 설치되어 있는지 확인하세요:
# !pip install transformers pandas

# 모델과 토크나이저 초기화
model_name = "snunlp/KR-Medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


# 감성 분석을 수행하기 전에 텍스트를 처리하는 함수
def preprocess_text(text, max_length=512):
    # NaN 값 또는 None을 건너뛰기
    if pd.isna(text) or text is None:
        return None
    # 텍스트를 최대 허용 길이로 자르기
    tokens = tokenizer.encode(text, add_special_tokens=True, max_length=max_length, truncation=True)
    # 토큰을 다시 텍스트로 변환
    return tokenizer.decode(tokens)
# 각 파일에 대해 감성 분석을 수행하고 JSON으로 결과를 저장하는 함수
def perform_sentiment_analysis(file_path, columns, file_type='csv', result_file_path=None):
    # 파일 타입에 따라 다르게 읽기
    if file_type == 'csv':
        df = pd.read_csv(file_path)
    elif file_type == 'json':
        df = pd.read_json(file_path)
    
    print(df.head())  # 데이터의 상위 몇 줄을 출력하여 확인
    
    # 결과 파일 이름 설정
    if result_file_path is None:
        result_file_path = file_path.split('.')[0] + '_sentiment_results.json'
    
    sentiment_results = []  # 감성 분석 결과를 저장할 리스트

    # 각 행과 지정된 컬럼에 대한 감성 분석을 수행
    for _, row in df.iterrows():
        for column in columns:
            text = preprocess_text(row.get(column, ''))
            if text:  # 수정된 부분: None이 아닌 경우에만 분석 수행
                try:
                    sentiment = nlp(text)[0]
                    sentiment_results.append({
                        "text": text,
                        "column": column,
                        "sentiment_label": sentiment['label'],
                        "sentiment_score": sentiment['score']
                    })
                except Exception as e:
                    print(f"텍스트 처리 오류: {text}. 오류: {e}")
    
    # 결과를 JSON 파일로 저장
    with open(result_file_path, 'w', encoding='utf-8') as file:
        json.dump(sentiment_results, file, ensure_ascii=False, indent=4)
    
    print(f"결과가 {result_file_path}에 저장되었습니다")

# 각 파일에 대해 함수를 호출합니다.
# 각 파일에 대해 함수를 호출합니다. 경로를 정확하게 지정하세요.
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/combined_data.csv', ['title', 'contents'])
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/inti_op.csv', ['text'])
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/naver_news.json', ['title', 'description'], file_type='json')
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/youtube_comments.csv', ['comment'])
perform_sentiment_analysis('/Users/jeonghyeonjeong/Documents/bigpl_team/data/youtube_videos.csv', ['video_title', 'video_des'])




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-Medium and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    id                                    Title  \
0  201                        김포 서울편입은 흐지부지 된거냐   
1  202                       서울에도 메트로개념 하면 안되려나   
2  203                  김포 서울 편입이슈가 한주도 안되서 잊혀짐   
3  204                      김포 서울 편입되면 얻게 되는 효과   
4  205  조경태 “메가시티는 제2새마을운동...국가부채 1000조, 뭉쳐야 산다   

                                           Post Link  \
0  https://gall.dcinside.com/board/view/?id=drama...   
1  https://gall.dcinside.com/mgallery/board/view/...   
2  https://gall.dcinside.com/mgallery/board/view/...   
3  https://gall.dcinside.com/board/view/?id=immov...   
4  https://gall.dcinside.com/mgallery/board/view/...   

                                            Contents  
0                                         갑자기 조용해진 듯  
1  요즘 서울 확장 얘기 많던데 서울"시"로 편입하기보다는 메트로벤쿠버나 GTA, 혹은...  
2              총선까지 한참 남았는데 병신들만 남아있는게 아닌가 생각이 된다 ㄹㅇ  
3  1. 각종 기피시설 김포 이전2. 김포집값이 서울아파트 평균값을 떨어뜨린다3. 서울...  
4  (주간조선)조경태 “메가시티는 제2 새마을운동...국가부채 1000조, 뭉쳐야 산다...  
결과가 /Users/jeonghyeonjeong

텍스트 처리 오류: [CLS] 롬 < a href = " https : / / www. youtube. com / watch? v = 78OkXyP4ri0 & amp ; t = 3m23s " > 3 : 23 < / a > < br > 이는 모든 사람이 죄를 지었으므로 < br > 하나님의 영광에 이르지 못하다가. < br > < br > 롬 < a href = " https : / / www. youtube. com / watch? v = 78OkXyP4ri0 & amp ; t = 6m23s " > 6 : 23 < / a > < br > 죄의 삯은 사망이요. < br > < br > 죄사함을 받지 못하시면 영원토록 < br > 불타는 지옥에서 고통받습니다 < br > < br > 행 < a href = " https : / / www. youtube. com / watch? v = 78OkXyP4ri0 & amp ; t = 8m37s " > 8 : 37 < / a > < br > 빌립이 말하기를 “ 만일 당신이 마음을 다하여 믿으면 합당하니라. ” 고 하니 그가 대답하여 말하기를 “ 나는 예수 그리스도가 하나님의 아들이신 것을 믿나이다. ” 라고 하더라 < br > < br > 롬10 : 9 < br > 네가 네 입으로 주 예수를 시인하고 또 하나님께서 그를 죽은 자들로부터 살리신 것을 네 마음에 믿으면 구원을 받으리라. < br > < br > 고전 15 : 3 - 4 < br > 내가 받은 것을 먼저 너희에게 전달하였나니 이는 성경대로 그리스도께서 우리의 죄들로 인하여 죽으시고, 장사되셨다가 성경대로 셋째 날에 다시 살아나셔서 < br > < br > 엡2 : 8 - 9 < br > 너희가 믿음으로 말미암아 은혜로 구원을 받았으니 이것은 너희에게서 난 것이 아니요, 하나님의 선물이라. < br > 행위에서 난 것이 아니니 아무도 자랑하지 못하게 하려 하심이라. < br > < br > < br > & # 39 ; & # 39 [SEP]. 오류: The si

텍스트 처리 오류: [CLS] 롬 < a href = " https : / / www. youtube. com / watch? v = b4frzRhem7s & amp ; t = 3m23s " > 3 : 23 < / a > < br > 이는 모든 사람이 죄를 지었으므로 < br > 하나님의 영광에 이르지 못하다가. < br > < br > 롬 < a href = " https : / / www. youtube. com / watch? v = b4frzRhem7s & amp ; t = 6m23s " > 6 : 23 < / a > < br > 죄의 삯은 사망이요. < br > < br > 죄사함을 받지 못하시면 영원토록 < br > 불타는 지옥에서 고통받습니다 < br > < br > 행 < a href = " https : / / www. youtube. com / watch? v = b4frzRhem7s & amp ; t = 8m37s " > 8 : 37 < / a > < br > 빌립이 말하기를 “ 만일 당신이 마음을 다하여 믿으면 합당하니라. ” 고 하니 그가 대답하여 말하기를 “ 나는 예수 그리스도가 하나님의 아들이신 것을 믿나이다. ” 라고 하더라 < br > < br > 롬10 : 9 < br > 네가 네 입으로 주 예수를 시인하고 또 하나님께서 그를 죽은 자들로부터 살리신 것을 네 마음에 믿으면 구원을 받으리라. < br > < br > 고전 15 : 3 - 4 < br > 내가 받은 것을 먼저 너희에게 전달하였나니 이는 성경대로 그리스도께서 우리의 죄들로 인하여 죽으시고, 장사되셨다가 성경대로 셋째 날에 다시 살아나셔서 < br > < br > 엡2 : 8 - 9 < br > 너희가 믿음으로 말미암아 은혜로 구원을 받았으니 이것은 너희에게서 난 것이 아니요, 하나님의 선물이라. < br > 행위에서 난 것이 아니니 아무도 자랑하지 못하게 하려 하심이라. < br > < br > < br > & # 39 ; & # 39 ; & # [SEP]. 오류: 

텍스트 처리 오류: [CLS] 댓꿀쇼 811회 하이라이트 < br > < br > - 꿀단지 [UNK] < br > < br > [UNK] 시작 < a href = " https : / / www. youtube. com / watch? v = o8EO _ ETJkVY & amp ; t = 16m42s " > 16 : 42 < / a > < br > < br > < a href = " https : / / www. youtube. com / watch? v = o8EO _ ETJkVY & amp ; t = 1m24s " > 1 : 24 < / a > 김현정의 뉴스쇼가 행운인 장성철 < br > < a href = " https : / / www. youtube. com / watch? v = o8EO _ ETJkVY & amp ; t = 5m21s " > 5 : 21 < / a > 김포 때문에 피곤한 하트피디? < br > < a href = " https : / / www. youtube. com / watch? v = o8EO _ ETJkVY & amp ; t = 10m27s " > 10 : 27 < / a > 정의당 이야기를 못해 서러운 박원석 < br > < a href = " https : / / www. youtube. com / watch? v = o8EO _ ETJkVY & amp ; t = 11m32s " > 11 : 32 < / a > 김포가 서울이 될 수 있는가 < br > < br > < a href = " https : / / www. youtube. com / watch? v = o8EO _ ETJkVY & amp ; t = 15m54s " > 15 : 54 < / a > 오프닝 < br > < a href = " h [SEP]. 오류: The size of tensor a (514) must match the size of tensor b (512) at non-singleton dimension 1
텍스트 처리 오류: [CLS] <

텍스트 처리 오류: [CLS] 롬 < a href = " https : / / www. youtube. com / watch? v = oa5h7Ugpf74 & amp ; t = 3m23s " > 3 : 23 < / a > < br > 이는 모든 사람이 죄를 지었으므로 < br > 하나님의 영광에 이르지 못하다가. < br > < br > 롬 < a href = " https : / / www. youtube. com / watch? v = oa5h7Ugpf74 & amp ; t = 6m23s " > 6 : 23 < / a > < br > 죄의 삯은 사망이요. < br > < br > 죄사함을 받지 못하시면 영원토록 < br > 불타는 지옥에서 고통받습니다 < br > < br > 행 < a href = " https : / / www. youtube. com / watch? v = oa5h7Ugpf74 & amp ; t = 8m37s " > 8 : 37 < / a > < br > 빌립이 말하기를 “ 만일 당신이 마음을 다하여 믿으면 합당하니라. ” 고 하니 그가 대답하여 말하기를 “ 나는 예수 그리스도가 하나님의 아들이신 것을 믿나이다. ” 라고 하더라 < br > < br > 롬10 : 9 < br > 네가 네 입으로 주 예수를 시인하고 또 하나님께서 그를 죽은 자들로부터 살리신 것을 네 마음에 믿으면 구원을 받으리라. < br > < br > 고전 15 : 3 - 4 < br > 내가 받은 것을 먼저 너희에게 전달하였나니 이는 성경대로 그리스도께서 우리의 죄들로 인하여 죽으시고, 장사되셨다가 성경대로 셋째 날에 다시 살아나셔서 < br > < br > 엡2 : 8 - 9 < br > 너희가 믿음으로 말미암아 은혜로 구원을 받았으니 이것은 너희에게서 난 것이 아니요, 하나님의 선물이라. < br > 행위에서 난 것이 아니니 아무도 자랑하지 못하게 하려 하심이라. < br > < br > < br > & # 39 ; & # 39 [SEP]. 오류: The si

In [4]:
import json

# 파일 경로들을 리스트에 저장
file_paths = [
    '/Users/jeonghyeonjeong/Documents/bigpl_team/data/감성분석/combined_data_sentiment_results.json',
    '/Users/jeonghyeonjeong/Documents/bigpl_team/data/감성분석/inti_op_sentiment_results.json',
    '/Users/jeonghyeonjeong/Documents/bigpl_team/data/감성분석/naver_news_sentiment_results.json',
    '/Users/jeonghyeonjeong/Documents/bigpl_team/data/감성분석/youtube_comments_sentiment_results.json',
    '/Users/jeonghyeonjeong/Documents/bigpl_team/data/감성분석/youtube_videos_sentiment_results.json'
]

# 모든 데이터를 저장할 리스트
combined_data = []

# 각 파일을 읽고 내용을 combined_data에 추가
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        combined_data.extend(data)

# 합쳐진 데이터를 새로운 JSON 파일로 저장
combined_file_path = '/Users/jeonghyeonjeong/Documents/bigpl_team/data/감성분석/combined_sentiment_results.json'
with open(combined_file_path, 'w', encoding='utf-8') as file:
    json.dump(combined_data, file, ensure_ascii=False, indent=4)

print(f"합쳐진 데이터가 {combined_file_path}에 저장되었습니다.")


합쳐진 데이터가 /Users/jeonghyeonjeong/Documents/bigpl_team/data/감성분석/combined_sentiment_results.json에 저장되었습니다.
